In [110]:
import requests
import pandas as pd
import json
import math
import requests
import urllib.parse
from datetime import datetime, timedelta

In [111]:
# formatting on dates
date_interval = "{0},{1}".format(datetime.now().date() - timedelta(days=2),datetime.now().date() + timedelta(days=1))

In [112]:
# handling the cookies 
def load_cookies():
	headers = {
	    'authority': 'www.barchart.com',
	    'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
	    'sec-ch-ua-mobile': '?0',
	    'sec-ch-ua-platform': '"Windows"',
	    'upgrade-insecure-requests': '1',
	    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36',
	    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
	    'sec-fetch-site': 'none',
	    'sec-fetch-mode': 'navigate',
	    'sec-fetch-user': '?1',
	    'sec-fetch-dest': 'document',
	    'accept-language': 'en-GB,en;q=0.9',
	}
	response = requests.get('https://www.barchart.com/options/volatility-percent-change/increase', headers=headers)
	return response.cookies

def get_parent(url,params,ignore,col_names,pagination):
	cookies = load_cookies()
	headers = {
	    'authority': 'www.barchart.com',
	    'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
	    'accept': 'application/json',
	    'x-xsrf-token': urllib.parse.unquote(cookies.get('XSRF-TOKEN')),
	    'sec-ch-ua-mobile': '?0',
	    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36',
	    'sec-ch-ua-platform': '"Windows"',
	    'sec-fetch-site': 'same-origin',
	    'sec-fetch-mode': 'cors',
	    'sec-fetch-dest': 'empty',
	    'referer': url,
	    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
	    'cookie': ';'.join(["{0}={1}".format(each_cookie[0],urllib.parse.unquote(each_cookie[1])) for each_cookie in cookies.items()])
			}

	# parsing data into pandas
	final_data_list = []
	response = requests.get(url, headers=headers, params=params)
	total_pages = math.ceil(json.loads(response.content).get("total") / 100)
	
	for page in range(1,total_pages+1):
		new_params = []
		for each_param in params:
			if 'page' != each_param[0]:
				new_params.append(each_param)
			else:
				new_params.append(('page', str(page)))
		params = new_params
		response = requests.get(url, headers=headers, params=params)
		final_data_list.extend(json.loads(response.content).get("data"))
		if not pagination:
			break
	final_data_list = [{key:val for key,val in row.items() if key not in ignore} for row in final_data_list]
	data_frame = pd.DataFrame(final_data_list)
	data_frame.columns = col_names
	return data_frame

**1) Most Active Stock Options**
<br>obtain latest data from: [barchart.com](https://www.barchart.com/options/most-active/stocks?orderBy=optionsTotalVolume&orderDir=desc)

In [113]:
# Most Active Stock Options

params = [
  ('list', 'options.mostActive.us'),
  ('fields', 'symbol,symbolType,symbolName,hasOptions,lastPrice,priceChange,percentChange,optionsImpliedVolatilityRank1y,optionsTotalVolume,optionsPutVolumePercent,optionsCallVolumePercent,optionsPutCallVolumeRatio,tradeTime,symbolCode'),
  ('orderBy', 'optionsTotalVolume'),
  ('orderDir', 'desc'),
  ('between(lastPrice,.10,)', ''),
  ('between(tradeTime,{0})'.format(date_interval), ''),
  ('meta', 'field.shortName,field.type,field.description'),
  ('hasOptions', 'true'),
  ('page', '1'),
  ('limit', '100'),
  ('raw', '1')]

high_vol_stock_options = get_parent('https://www.barchart.com/proxies/core-api/v1/quotes/get',params,['symbolCode','raw','symbolType','hasOptions'],
		                    ["Symbol","Name","Last","Change","%Chg","IV Rank","Options Vol" ,"% Put","% Call","Put/Call Vol","Time"],True)

In [115]:
# Highest vol stock options clean up
high_vol_stock_options['Time'] = high_vol_stock_options['Time'].astype('datetime64')
to_num = ['Last','IV Rank', 'Options Vol', '% Put', '% Call', 'Put/Call Vol']
high_vol_stock_options[to_num] = high_vol_stock_options[to_num].replace(r'\D+', '', regex=True)
high_vol_stock_options[to_num] = high_vol_stock_options[to_num].astype('float')
high_vol_stock_options[['Last', 'Options Vol', 'IV Rank', 'Put/Call Vol']] = high_vol_stock_options[['Last', 'Options Vol', 'IV Rank', 'Put/Call Vol']]/100
high_vol_stock_options[['% Put', '% Call']] = high_vol_stock_options[['% Put', '% Call']]/10

In [116]:
high_vol_stock_options.head()

,Symbol,Name,Last,Change,%Chg,IV Rank,Options Vol,% Put,% Call,Put/Call Vol,Time
0,TSLA,Tesla Inc,775.22,-0.26,-0.03%,21.21,14626.21,58.2,41.8,1.39,2021-10-01
1,AAPL,Apple Inc,142.65,+1.15,+0.81%,27.42,13588.44,32.2,67.8,0.47,2021-10-01
2,AMC,AMC Entertainment Holdings Inc,38.46,+0.40,+1.05%,2.92,6997.80,24.6,75.4,0.33,2021-10-01
3,MRK,Merck & Company,81.40,+6.29,+8.37%,51.37,3958.63,20.9,79.1,0.26,2021-10-01
4,AMZN,Amazon.com Inc,3283.26,-1.78,-0.05%,34.95,3506.40,42.0,58.0,0.72,2021-10-01


**2) Most Active ETF Options**
<br>obtain latest data from: [barchart.com](https://www.barchart.com/options/most-active/etfs?orderBy=optionsTotalVolume&orderDir=desc)

In [60]:
# Most Active ETFs

params = [
  ('list', 'options.mostActive.etf'),
	('fields', 'symbol,symbolType,symbolName,hasOptions,lastPrice,priceChange,percentChange,optionsImpliedVolatilityRank1y,optionsTotalVolume,optionsPutVolumePercent,optionsCallVolumePercent,optionsPutCallVolumeRatio,tradeTime,symbolCode'),
	('orderBy', 'optionsTotalVolume'),
	('orderDir', 'desc'),
	('between(lastPrice,.10,)', ''),
	('between(tradeTime,{0})'.format(date_interval), ''),
	('meta', 'field.shortName,field.type,field.description'),
	('hasOptions', 'true'),
	('page', '1'),
	('limit', '100'),
	('raw', '1')]

high_vol_etf_options = get_parent('https://www.barchart.com/proxies/core-api/v1/quotes/get',params,['symbolCode','raw','symbolType','hasOptions'],
		                  ["Symbol","Name","Last","Change","%Chg","IV Rank","Options Vol" ,"% Put","% Call","Put/Call Vol","Time"],True)

In [118]:
# Highest vol etr options clean up
high_vol_etf_options['Time'] = high_vol_etf_options['Time'].astype('datetime64')
to_num = ['Last','IV Rank', 'Options Vol', '% Put', '% Call', 'Put/Call Vol']
high_vol_etf_options[to_num] = high_vol_etf_options[to_num].replace(r'\D+', '', regex=True)
high_vol_etf_options[to_num] = high_vol_etf_options[to_num].astype('float')
high_vol_etf_options[['Last', 'Options Vol', 'IV Rank', 'Put/Call Vol']] = high_vol_stock_options[['Last', 'Options Vol', 'IV Rank', 'Put/Call Vol']]/100
high_vol_etf_options[['% Put', '% Call']] = high_vol_etf_options[['% Put', '% Call']]/10

In [119]:
high_vol_etf_options.head()

,Symbol,Name,Last,Change,%Chg,IV Rank,Options Vol,% Put,% Call,Put/Call Vol,Time
0,SPY,S&P 500 SPDR,7.7522,+5.10,+1.19%,0.2121,146.2621,54.7,45.3,0.0139,2021-10-01
1,QQQ,Nasdaq QQQ Invesco ETF,1.4265,+2.22,+0.62%,0.2742,135.8844,57.1,42.9,0.0047,2021-10-01
2,IWM,Russell 2000 Ishares ETF,0.3846,+3.53,+1.61%,0.0292,69.9780,73.0,27.0,0.0033,2021-10-01
3,XLE,S&P 500 Energy Sector SPDR,0.8140,+1.75,+3.36%,0.5137,39.5863,14.2,85.8,0.0026,2021-10-01
4,HYG,High Yield Corp Bond Ishares Iboxx $ ETF,32.8326,-0.08,-0.09%,0.3495,35.0640,86.1,13.9,0.0072,2021-10-01


**3) Highest Implied Volatility Options**
<br>obtain latest data from: [barchart.com](https://www.barchart.com/options/highest-implied-volatility)

In [62]:
# Highest implied volatility table

params = [
  ('fields', 'symbol,baseSymbol,baseLastPrice,baseSymbolType,symbolType,strikePrice,expirationDate,daysToExpiration,bidPrice,midpoint,askPrice,lastPrice,volume,openInterest,volumeOpenInterestRatio,volatility,tradeTime,symbolCode,hasOptions'),
  ('orderBy', 'volatility'),
  ('baseSymbolTypes', 'stock'),
  ('between(lastPrice,.10,)', ''),
  ('between(daysToExpiration,15,)', ''),
  ('between(tradeTime,{0})'.format(date_interval), ''),
  ('orderDir', 'desc'),
  ('between(volatility,60,)', ''),
  ('limit', '200'),
  ('between(volume,500,)', ''),
  ('between(openInterest,100,)', ''),
  ('in(exchange,(AMEX,NASDAQ,NYSE))', ''),
  ('meta', 'field.shortName,field.type,field.description'),
  ('hasOptions', 'true'),
  ('raw', '1')]
  
highest_volitility = get_parent('https://www.barchart.com/proxies/core-api/v1/options/get',params,['symbol','baseSymbolType','raw'],
		                ["Symbol","Price","Type","Strike","Exp Date","DTE","Bid","Midpoint","Ask","Last","Volume","Open Int","Vol/OI","IV","Last Trade"],False)

In [63]:
highest_volitility.head()

,Symbol,Price,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Volume,Open Int,Vol/OI,IV,Last Trade
0,NXTD,0.4020,Call,0.5000,11/19/21,49,0.1000,0.1300,0.1500,0.1400,"2,325","1,519",1.53,295.35%,10/01/21
1,PROG,1.7900,Call,2.0000,11/19/21,49,0.5500,0.5800,0.6000,0.6000,"7,415","1,965",3.77,262.77%,10/01/21
2,PROG,1.7900,Call,1.5000,11/19/21,49,0.7000,0.7300,0.7500,0.7300,"3,904","7,206",0.54,247.26%,10/01/21
3,XENE,15.60,Put,7.50,11/19/21,49,0.65,1.03,1.40,0.70,"3,036","2,181",1.39,240.68%,09/30/21
4,BBIG,6.14,Put,20.00,01/21/22,112,15.00,15.15,15.30,15.00,"2,145","5,634",0.38,237.66%,09/30/21


In [64]:
# Highest IV data clean up
highest_volitility[['Exp Date', 'Last Trade']] = highest_volitility[['Exp Date', 'Last Trade']].astype('datetime64')
to_num = ['Price', 'Strike', 'DTE', 'Bid', 'Midpoint', 'Ask', 'Last',	'Volume',	'Open Int',	'IV', 'Vol/OI']
highest_volitility['Volume'] = highest_volitility['Volume'].str.replace(',', '')
highest_volitility['Open Int'] = highest_volitility['Open Int'].str.replace(',', '')
highest_volitility['IV'] = highest_volitility['IV'].str.replace('%', '')
highest_volitility[to_num] = highest_volitility[to_num].astype('float')

**3) Highest Implied Volatility Options Screener**
<br>Screening Criteria:
*   Implied Volatility > 100%
*   Day to expiry within 21 days
*   Stock price < USD 100 per share





In [121]:
# screen for highest IV, within 21 days out for put options
high_iv_screener = highest_volitility.query('IV> 100 & DTE<22 & Price< 100')
high_iv_screener

,Symbol,Price,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Volume,Open Int,Vol/OI,IV,Last Trade
9,AMC,38.46,Call,95.0,2021-10-22,21.0,0.46,0.48,0.49,0.48,1574.0,4625.0,0.34,214.34,2021-10-01
12,DMYI,10.40,Call,20.0,2021-10-15,15.0,0.10,0.13,0.15,0.15,1563.0,8807.0,0.18,206.20,2021-09-30
23,BBIG,6.14,Call,7.0,2021-10-22,21.0,0.75,0.78,0.80,0.80,556.0,1845.0,0.30,189.65,2021-10-01
39,DMYI,10.40,Call,15.0,2021-10-15,15.0,0.25,0.30,0.35,0.30,862.0,7534.0,0.11,167.36,2021-09-30
45,DMYI,10.40,Call,14.0,2021-10-15,15.0,0.40,0.43,0.45,0.40,641.0,3738.0,0.17,163.39,2021-09-30
52,DMYI,10.40,Call,12.5,2021-10-15,15.0,0.50,0.55,0.60,0.60,4511.0,7527.0,0.60,153.06,2021-09-30
62,AMC,38.46,Call,60.0,2021-10-22,21.0,0.85,0.89,0.92,0.89,680.0,1340.0,0.51,149.97,2021-10-01
81,DMYI,10.40,Call,11.0,2021-10-15,15.0,0.80,0.88,0.95,0.93,1622.0,2736.0,0.59,139.90,2021-09-30
86,DMYI,10.40,Call,10.0,2021-10-15,15.0,1.25,1.30,1.35,1.34,1971.0,5948.0,0.33,137.28,2021-09-30
94,MNKD,4.52,Call,5.0,2021-10-22,21.0,0.35,0.40,0.45,0.34,2008.0,3121.0,0.64,134.65,2021-10-01


In [125]:
# put tickers to a list
highest_iv_list = high_iv_screener['Symbol'].unique().tolist()